In [1]:
%load_ext autoreload
%autoreload 2

In [63]:
from sklearn.metrics import make_scorer
from MIDIComposingAI.create_dataset import *
from MIDIComposingAI.get_back_data import *
from MIDIComposingAI.utils import piano_roll_to_pretty_midi
import joblib
import pretty_midi
from scipy.stats import entropy
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import make_scorer
from sklearn.preprocessing import StandardScaler, Normalizer, MinMaxScaler
from sklearn.model_selection import train_test_split

In [57]:
file = joblib.load('../raw_data/pretty_midi/(Day Dream) Prayer')
file2 = pretty_midi.PrettyMIDI('../raw_data/1.mid')

In [ ]:
def pattern_recognition(acc, mel):

    # Create list of notes for each instant in accompaniment's piano roll
    acc_notes = []
    
    for instant in acc.T:
        notes = []
        for note in instant:
            if note > 0:
                notes.append(list(instant).index(note))
        if not notes:
            notes.append(0)
        acc_notes.append(notes)
    
    # Take the mean note of each instant
    mean_notes_acc = [(np.sum(notes) / len(notes)) for notes in acc_notes]
    
    # Get only the notes (not the velocities)
    melody_notes = mel[:500]
    
    
    # Create the pattern for accompaniment
    acc_passed_note = []
    pattern_acc = [0]
    
    for note in mean_notes_acc:
        if acc_passed_note:
            if note != acc_passed_note[-1]:
                relative_note = note - acc_passed_note[-1]
                pattern_acc.append(relative_note)
            else:
                pattern_acc.append(pattern_acc[-1])
        acc_passed_note.append(note)
    
    # Create the pattern for melody
    mel_passed_note = []
    pattern_mel = [0]
    
    for note in melody_notes:
        if mel_passed_note:
            if note != mel_passed_note[-1]:
                relative_note = note - mel_passed_note[-1]
                pattern_mel.append(relative_note)
            else:
                pattern_mel.append(pattern_mel[-1])
        mel_passed_note.append(note)
        
    return pattern_acc, pattern_mel

def custom_metric(acc, pred):
    
    # Get the pitch pattern for both accompaniment and melody
    pattern_acc, pattern_mel = pattern_recognition(acc, pred)
    
    # Compute the mean of velocities for both accompaniment and melody
    mean_vel_acc = np.mean([np.mean(instant) for instant in acc.T])
    mean_vel_pred = np.mean(pred[500:])
    
    # Compute the diff beetween the two velocities mean
    velocity_diff = abs(mean_vel_acc - mean_vel_pred)
    
    # Compute the "diff pattern" beetween accompaniment and melody
    diff_pattern = np.array([acc - mel for acc, mel in zip(pattern_acc, pattern_mel)]).reshape(-1, 1)
    
    # Compute the entropy of the diff pattern
    entropy_score = entropy(diff_pattern, axis=1)
    
    # Compute the final score
    score = velocity_diff + entropy_score
    
    return score

### Compute the score within a grid search

In [86]:
X, y = create_simple_dataset(file)
X_reshaped = X.reshape((X.shape[0], -1))
y = y.reshape((y.shape[0], -1))

In [108]:
tree = DecisionTreeRegressor()

grid = {
    'criterion':               ["squared_error","friedman_mse","absolute_error","poisson"],
    'max_depth':               [None, 2, 12, 128],
    'min_samples_split':       [2, 3, 5, 10],
    'min_samples_leaf':        [1, 2, 3, 4],
    'min_weight_fraction_leaf':[0.0, 0.2, 0.4, 0.5],
    'max_leaf_nodes':          [None, 128, 12, 2],
    # 'min_impurity_decrease':   [0.0, 0.2, 0.5, 0.8],
}

In [110]:
params = [{'criterion':crit,
           'max_depth':max_d,
           'min_samples_split':min_ss,
           'min_samples_leaf':min_sl,
           'min_weight_fraction_leaf':min_w,
           'max_leaf_nodes':max_l}
           # 'min_impurity_decrease':min_i}
          for crit in grid['criterion']
          for max_d in grid['max_depth']
          for min_ss in grid['min_samples_split']
          for min_sl in grid['min_samples_leaf']
          for min_w in grid['min_weight_fraction_leaf']
          for max_l in grid['max_leaf_nodes']]
          # for min_i in grid['min_impurity_decrease']]

In [111]:
len(params)

4096

In [87]:
X_train, X_test, X_reshaped_train, X_reshaped_test, y_train, y_test = train_test_split(X, X_reshaped, y, test_size=0.2)


In [75]:
y[0].shape

(1000,)

In [156]:
params_and_scores = []

for i, param in enumerate(params):
    
    tree = DecisionTreeRegressor(**param)
    tree.fit(X_reshaped_train, y_train)
    predictions = tree.predict(X_reshaped_test)
    scores = [custom_metric(test, pred) for test, pred in zip(X_test, predictions)]
    score = np.mean(scores[0])
    params_and_scores.append({'params':param, 'score':score})
    if i % 50 == 0:
        print(f'{i+1} done')

[[  0.]
 [  0.]
 [  0.]
 [  0.]
 [  0.]
 [  0.]
 [  0.]
 [  0.]
 [  0.]
 [  0.]
 [  0.]
 [  0.]
 [  0.]
 [  0.]
 [  0.]
 [  0.]
 [  0.]
 [  0.]
 [  0.]
 [  0.]
 [  0.]
 [  0.]
 [  0.]
 [  0.]
 [  0.]
 [  0.]
 [  0.]
 [  0.]
 [  0.]
 [  0.]
 [  0.]
 [  0.]
 [  0.]
 [  0.]
 [  0.]
 [  0.]
 [  0.]
 [  0.]
 [  0.]
 [  0.]
 [  0.]
 [  0.]
 [  0.]
 [  0.]
 [  0.]
 [  0.]
 [  0.]
 [  0.]
 [  0.]
 [  0.]
 [  0.]
 [  0.]
 [  0.]
 [  0.]
 [  0.]
 [  0.]
 [  0.]
 [  0.]
 [  0.]
 [  0.]
 [  0.]
 [  0.]
 [  0.]
 [  0.]
 [  0.]
 [  0.]
 [  0.]
 [  0.]
 [  0.]
 [  0.]
 [  0.]
 [  0.]
 [  0.]
 [  0.]
 [  0.]
 [  0.]
 [  0.]
 [  0.]
 [  0.]
 [  0.]
 [  0.]
 [  0.]
 [  0.]
 [  0.]
 [  0.]
 [  0.]
 [  0.]
 [  0.]
 [  0.]
 [  0.]
 [  0.]
 [  0.]
 [  0.]
 [  0.]
 [  0.]
 [  0.]
 [  0.]
 [  0.]
 [  0.]
 [  0.]
 [  0.]
 [  0.]
 [  0.]
 [  0.]
 [  0.]
 [  0.]
 [  0.]
 [  0.]
 [  0.]
 [  0.]
 [  0.]
 [  0.]
 [  0.]
 [  0.]
 [  0.]
 [  0.]
 [  0.]
 [  0.]
 [  0.]
 [  0.]
 [  0.]
 [  0.]
 [  0.]
 [  0.]
 [  0.]


/home/voncq/.pyenv/versions/3.8.12/envs/MIDIComposingAI/lib/python3.8/site-packages/scipy/stats/_entropy.py:72: RuntimeWarning: invalid value encountered in true_divide
  pk = 1.0*pk / np.sum(pk, axis=axis, keepdims=True)


[[   0.]
 [   0.]
 [   0.]
 [   0.]
 [   0.]
 [   0.]
 [   0.]
 [   0.]
 [   0.]
 [   0.]
 [   0.]
 [   0.]
 [   0.]
 [   0.]
 [   0.]
 [   0.]
 [   0.]
 [   0.]
 [   0.]
 [   0.]
 [   0.]
 [   0.]
 [   0.]
 [   0.]
 [   0.]
 [   0.]
 [   0.]
 [   0.]
 [   0.]
 [   0.]
 [   0.]
 [   0.]
 [   0.]
 [   0.]
 [   0.]
 [   0.]
 [   0.]
 [   0.]
 [   0.]
 [   0.]
 [   0.]
 [   0.]
 [   0.]
 [   0.]
 [   0.]
 [   0.]
 [   0.]
 [   0.]
 [   0.]
 [   0.]
 [   0.]
 [   0.]
 [   0.]
 [   0.]
 [   0.]
 [   0.]
 [   0.]
 [   0.]
 [   0.]
 [   0.]
 [   0.]
 [   0.]
 [   0.]
 [   0.]
 [   0.]
 [   0.]
 [   0.]
 [   0.]
 [   0.]
 [   0.]
 [   0.]
 [   0.]
 [   0.]
 [   0.]
 [   0.]
 [   0.]
 [   0.]
 [   0.]
 [   0.]
 [   0.]
 [   0.]
 [   0.]
 [   0.]
 [   0.]
 [   0.]
 [   0.]
 [   0.]
 [   0.]
 [   0.]
 [   0.]
 [   0.]
 [   3.]
 [  64.]
 [  64.]
 [  64.]
 [  64.]
 [  64.]
 [  64.]
 [  64.]
 [  64.]
 [  64.]
 [  64.]
 [  64.]
 [ -69.]
 [ -69.]
 [ -69.]
 [ -69.]
 [-128.]
 [-128.]
 [-128.]
 [-128.]
 

/home/voncq/.pyenv/versions/3.8.12/envs/MIDIComposingAI/lib/python3.8/site-packages/scipy/stats/_entropy.py:72: RuntimeWarning: invalid value encountered in true_divide
  pk = 1.0*pk / np.sum(pk, axis=axis, keepdims=True)


[[   0.]
 [   0.]
 [   0.]
 [   0.]
 [   0.]
 [   0.]
 [   0.]
 [   0.]
 [   0.]
 [   0.]
 [   0.]
 [   0.]
 [   0.]
 [   0.]
 [   0.]
 [   0.]
 [   0.]
 [   0.]
 [   0.]
 [   0.]
 [   0.]
 [   0.]
 [   0.]
 [   0.]
 [   0.]
 [   0.]
 [   0.]
 [   0.]
 [   0.]
 [   0.]
 [   0.]
 [   0.]
 [   0.]
 [   0.]
 [   0.]
 [   0.]
 [   0.]
 [   0.]
 [   0.]
 [   0.]
 [   0.]
 [   0.]
 [   0.]
 [   0.]
 [   0.]
 [   0.]
 [   0.]
 [   0.]
 [   0.]
 [   0.]
 [   0.]
 [   0.]
 [   0.]
 [   0.]
 [   0.]
 [   0.]
 [   0.]
 [   0.]
 [   0.]
 [   0.]
 [   0.]
 [   0.]
 [   0.]
 [   0.]
 [   0.]
 [   0.]
 [   0.]
 [   0.]
 [   0.]
 [   0.]
 [   0.]
 [   0.]
 [   0.]
 [   0.]
 [   0.]
 [   0.]
 [   0.]
 [   0.]
 [   0.]
 [   0.]
 [   0.]
 [   0.]
 [   0.]
 [   0.]
 [   0.]
 [   0.]
 [   0.]
 [   0.]
 [   0.]
 [   0.]
 [   0.]
 [   3.]
 [  64.]
 [  64.]
 [  64.]
 [  64.]
 [  64.]
 [  64.]
 [  64.]
 [  64.]
 [  64.]
 [  64.]
 [  64.]
 [ -69.]
 [ -69.]
 [ -69.]
 [ -69.]
 [-128.]
 [-128.]
 [-128.]
 [-128.]
 

...

[[  0.        ]
 [  0.        ]
 [  0.        ]
 [  0.        ]
 [  0.        ]
 [  0.        ]
 [  0.        ]
 [  0.        ]
 [  0.        ]
 [  0.        ]
 [  0.        ]
 [  0.        ]
 [  0.        ]
 [  0.        ]
 [  0.        ]
 [  4.42857143]
 [  4.42857143]
 [  4.42857143]
 [  4.42857143]
 [  4.42857143]
 [  0.28571429]
 [  0.57142857]
 [  4.21428571]
 [  4.21428571]
 [  4.21428571]
 [  4.21428571]
 [  4.21428571]
 [  4.21428571]
 [  4.21428571]
 [  4.21428571]
 [  4.21428571]
 [  4.21428571]
 [  4.21428571]
 [  4.21428571]
 [  4.21428571]
 [  4.21428571]
 [  4.21428571]
 [  4.21428571]
 [  4.21428571]
 [  4.21428571]
 [  4.21428571]
 [  4.21428571]
 [  4.21428571]
 [  4.21428571]
 [  4.21428571]
 [  0.35714286]
 [  4.42857143]
 [  4.42857143]
 [  4.42857143]
 [  4.42857143]
 [  4.42857143]
 [  4.42857143]
 [  4.42857143]
 [  4.42857143]
 [  4.42857143]
 [  4.42857143]
 [  4.42857143]
 [  4.42857143]
 [  4.42857143]
 [  4.42857143]
 [  4.42857143]
 [  4.42857143]
 [  4.42

/home/voncq/.pyenv/versions/3.8.12/envs/MIDIComposingAI/lib/python3.8/site-packages/scipy/stats/_entropy.py:72: RuntimeWarning: invalid value encountered in true_divide
  pk = 1.0*pk / np.sum(pk, axis=axis, keepdims=True)


[[  0. ]
 [  0. ]
 [  0. ]
 [  0. ]
 [  0. ]
 [  0. ]
 [  0. ]
 [  0. ]
 [  0. ]
 [  6.9]
 [  6.9]
 [  6.9]
 [  6.9]
 [  6.9]
 [  6.9]
 [  6.9]
 [  6.9]
 [  6.9]
 [  6.9]
 [  6.9]
 [  6.9]
 [  6.9]
 [  6.9]
 [  6.9]
 [  6.9]
 [  6.9]
 [  0.3]
 [  0.3]
 [  0.4]
 [  0.4]
 [ -0.7]
 [ -0.7]
 [ -0.7]
 [ -0.7]
 [ -0.7]
 [ -0.7]
 [ -0.7]
 [  0.3]
 [  0.3]
 [  6.4]
 [  6.4]
 [  6.4]
 [  6.4]
 [  6.4]
 [  6.4]
 [  6.4]
 [  6.4]
 [  6.4]
 [  6.4]
 [  6.4]
 [  6.4]
 [  6.4]
 [  0.3]
 [  0.3]
 [ -0.3]
 [ -0.3]
 [ -0.3]
 [ -0.3]
 [ -0.3]
 [  6.9]
 [  6.9]
 [ -6.9]
 [ -6.9]
 [ -6.9]
 [ -6.9]
 [ -6.9]
 [ -6.9]
 [ -6.9]
 [ -6.9]
 [ -7.1]
 [ -7.1]
 [ -7.1]
 [ -7.1]
 [ -7.1]
 [  6.4]
 [  6.4]
 [  6.4]
 [  6.4]
 [  6.4]
 [-45.6]
 [-45.6]
 [-58.9]
 [-58.9]
 [-58.9]
 [-58.9]
 [-58.9]
 [-58.9]
 [-58.9]
 [-51.8]
 [-51.8]
 [-51.8]
 [-45.3]
 [-45.3]
 [-45.3]
 [-45.3]
 [-45.3]
 [-45.3]
 [-45.3]
 [-45.3]
 [-45.3]
 [-45.3]
 [-45.3]
 [ 63.7]
 [ 50.3]
 [ 50.3]
 [ 50.3]
 [ 50.3]
 [ 50.3]
 [ 63.7]
 [ 63.7]
 [ 63.7]
 

/home/voncq/.pyenv/versions/3.8.12/envs/MIDIComposingAI/lib/python3.8/site-packages/scipy/stats/_entropy.py:72: RuntimeWarning: invalid value encountered in true_divide
  pk = 1.0*pk / np.sum(pk, axis=axis, keepdims=True)


KeyboardInterrupt: 